# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'West Cottage',
 'AddressLine2': 'South Foreland Lighthouse',
 'AddressLine3': 'The Front',
 'AddressLine4': 'St Margarets Bay',
 'BusinessName': 'Mrs Knotts Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.492467848017,
 'FHRSID': 429991,
 'LocalAuthorityBusinessID': 'PI/000055200',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6HP',
 'RatingDate': '2019-05-02T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64700563bdde7d4e2206522d'),
 'geocode': {'latitude': '51.14061', 'longitude': '1.371894'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/429991',
            'rel': 'self'}],
 'meta': {'dataSource': None,
  

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [13]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
res = establishments.find(query)
for r in res:
    pprint(r)
    break

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('647009f1a54507f51b1969be'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType":"Restaurant/Cafe/Canteen"}
res = establishments.find(query)
print(res[0]["BusinessTypeID"])

1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [16]:
# Update the new restaurant with the correct BusinessTypeID
filter = {'BusinessName':'Penang Flavours'}
newvalues = { "$set": {'BusinessTypeID':1}}
establishments.update_one(filter,newvalues)

In [23]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
res = establishments.find(query)
pprint(res[0]["BusinessTypeID"])

1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [34]:
# Find how many documents have LocalAuthorityName as "Dover"
dovers = establishments.find( {'LocalAuthorityName': 'Dover'}).count()
print(dovers)

994


C:\Users\borna\AppData\Local\Temp\ipykernel_15240\428311089.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  dovers = establishments.find( {'LocalAuthorityName': 'Dover'}).count()


In [36]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_docs = {'LocalAuthorityName':'Dover'}
deleting = establishments.delete_many(dover_docs)
print(deleting.deleted_count, "documents deleted")

994 documents deleted


In [37]:
# Check if any remaining documents include Dover
dovers = establishments.find( {'LocalAuthorityName': 'Dover'}).count()
print(dovers)

0


C:\Users\borna\AppData\Local\Temp\ipykernel_15240\3483433717.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  dovers = establishments.find( {'LocalAuthorityName': 'Dover'}).count()


In [38]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('64700563bdde7d4e22065512'),
 'FHRSID': 289352,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6AB',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2015-08-27T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.188537', 'latitude': '51.08084'},
 'RightToReply': '',
 'Distance': 4591.912144538092,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [59]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({},[{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"},"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [54]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [60]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},[{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])

In [64]:
# Check that the coordinates and rating value are now numbers
res = establishments.find_one()
print("The type of latitude is: " + str(type(res['geocode']['latitude'])))
print("The type of longtitude is: " + str(type(res['geocode']['longitude'])))
print("The type of RatingValue is: " + str(type(res['RatingValue'])))

The type of latitude is: <class 'float'>
The type of longtitude is: <class 'float'>
The type of RatingValue is: <class 'int'>


In [66]:
mongo.close()